# Laboratorul 5 

## Introducere
Acest document prezintă o aplicație complexă dezvoltată în Python, concepută pentru procesarea și manipularea imaginilor folosind biblioteci moderne de grafică și editare vizuală. Aplicația oferă o interfață grafică modernă și intuitivă pentru manipularea imaginilor cu următoarele caracteristici principale:

1. Suport pentru diverse spații de culoare (RGB, Grayscale, HSV etc.)
2. Gestionarea și conversia între multiple formate de imagine (BMP, JPEG, PNG etc.)
3. Implementarea unor tehnici de editare a imaginilor, precum ajustarea luminozității, contrastului și aplicarea filtrelor
4. Instrumente pentru scalare și rotație eficientă a imaginilor
5. Interfață grafică prietenoasă și ușor de utilizat pentru manipularea diverselor funcționalități

Proiectul demonstrează aplicarea practică a conceptelor fundamentale de procesare a imaginilor, oferind o platformă utilă pentru explorarea tehnicilor de editare și analiză vizuală.

## Structura Codului

Aplicatia este organizata in urmatoarele sectiuni principale:

1. Instalarea bibliotecilor și modulelor necesare
2. Declararea variabilelor globale ale aplicației de laborator
3. Funcția pentru schimbarea spațiului de culoare folosit
4. Funcțiile pentru încărcarea, setarea extensiei și salvarea imaginilor
5. Funcția pentru actualizarea imaginii locale cu modificările efectuate
6. Funcțiile pentru editarea imaginii
7. Funcțiile pentru transformarea imaginii
8. Funcțiile pentru instanțierea UI-ului
9. Concluzii și bibliografie

## 1. Instalarea bibliotecilor și modulelor necesare
Din pricina faptului că sunt folosite anumite module care nu sunt prevăzute în mod implicit în Python Vanilla, va trebui să le importăm, lucru care se poate realiza prin rularea codului de mai jos

## 1.1 Instalarea bibliotecilor

In [88]:
# Este de preferat ca această porțiune de cod să se execute numai în cazul în care apar probleme în rularea codului din secțiunile următoare.
#!pip install opencv-python PyQt5 numpy

## 1.2 Importul modulelor necesare pentru laborator

In [89]:
# Această celulă trebuie rulată NEAPĂRAT înainte de începerea laboratorului propriu-zis
# Cu ajutorul acestor module și funcții vom realiza diferite operații de prelucrare a imaginilor
import sys
import os
import cv2
import numpy as np
import math
from PyQt5.QtWidgets import *
from PyQt5.QtGui import *
from PyQt5.QtCore import *
from functools import partial
from scipy.signal import convolve2d

## 1.3 Rolul modulelor pe care le vom folosi
Python este un limbaj de programare care se folosește foarte mult de module, așa că este momentul potrivit pentru a explica în mare ce face fiecare dintre modulele pe care le-am importat în blocul de deasupra.
Astfel:
* **sys** și **os** – Are ca rol manipularea sistemului și facilitează interacțiunea cu fișierele/directoarele sistemului de calcul.
* **cv2** – OpenCV este unul dintre tool-urile principale care ne vor ajuta la procesarea de imagini.
* **numpy as np** – Numpy este folosit pentru a facilita manipularea mult mai eficientă a matricelor și datelor numerice.
* **PyQt5.QtWidgets** – Acest modul ne ajută la realizarea componentelor pentru GUI.
* **PyQt5.QtGui** – Modulul ne ajută la gestionarea imaginilor, fonturilor și pictogramelor în cadrul GUI-ului.
* **PyQt5.QtCore** – Acest modul prezintă funcționalități de bază precum gestionarea evenimentelor și a thread-urilor în cadrul programului.
* **functools** -  Acest modul fixează un argument al unei funcții, ca atunci când o vom apela mai târziu, acel argument să rămână presetat.
* **scypy.signal** - Acest modul ne oferă funcții pentru prelucrarea semnalelor și operații matematice utile în procesarea imaginilor cum ar fi convoluția și filtrele

## 2. Declararea variabilelor globale ale aplicației de laborator

În Jupyter Notebook, variabilele globale au mai multe roluri, iar printre cele mai importante trebuie să menționăm:
* Asigură o lizibilitate mai mare codului scris
* Asgură persistența datelor de la o celulă la alta, permițând menținerea stării aplicației chiar dacă acestea sunt rulate în ordine diferită.
* Asigură o capacitate de modularitate mai crescută

In [90]:
# Variabile globale pentru starea aplicației

current_image=None 
original_image=None
current_path=None
app=None
main_window=None
image_label=None
brightness_slider=None
contrast_slider=None
filter_combo=None
smoothing_slider=None
scale_combo=None
rotation_combo=None
format_combo=None
tabs=None
format_tab=None
selected_format=None

## 3. Funcția pentru schimbarea spațiului de culoare folosit

In [91]:
def change_color_space(index):
    """Schimbă spațiul de culoare al imaginii."""
    global current_image, original_image, computed, using_cv2
    
    # Verifică dacă există o imagine încărcată
    if original_image is None:
        return
        
    # Resetează imaginea curentă la cea originală pentru a aplica noi transformări
    current_image=original_image.copy()
    
    match index:
        case 0:  # Original
            pass
            
        case 1:  # Grayscale computed
            # Calculează manual conversia la grayscale folosind formula ponderată RGB
            current_image=np.array(0.299*original_image[:,:,2]+0.587*original_image[:,:,1]+0.114*original_image[:,:,0]).astype(np.uint8)
            
            # Salvează versiunea calculată pentru comparații ulterioare
            computed=current_image.copy()
            
        case 2:  # Grayscale OpenCV
            # Convertește la grayscale folosind funcția OpenCV
            current_image=cv2.cvtColor(current_image,cv2.COLOR_BGR2GRAY)
            
            # Salvează versiunea OpenCV pentru comparații ulterioare
            using_cv2=current_image.copy()
            
        case 3:  # Comparație între grayscale calculat manual și OpenCV
            # Calculează diferența absolută între cele două conversii
            diff=np.abs(using_cv2-computed)
            
            # Normalizează diferența pentru vizualizare
            diff = cv2.normalize(diff,None,0,255,cv2.NORM_MINMAX)
            current_image=diff
            
            # Verifică dacă există diferențe între cele două metode
            if(np.sum(current_image)):
                print("Cele 2 conversii nu sunt perfect identice")
            else:
                print("Cele 2 conversii sunt identice")
                
        case 4:  # HSV OpenCV
            current_image=cv2.cvtColor(current_image,cv2.COLOR_BGR2HSV)
            
        case 5:  # HSV calculat manual
            # Convertește valorile BGR la interval [0,1]
            hsv_computed=current_image.astype(np.float32)/255.0
            r,g,b=original_image[...,0],original_image[...,1],original_image[...,2]
            
            # Calculează valorile maxime și minime pentru fiecare pixel
            c_max=np.max(original_image,axis=-1)
            c_min=np.min(original_image,axis=-1)
            delta=c_max-c_min

            # Calculează componenta H (Hue)
            h=np.zeros_like(c_max,dtype=np.float32)
            mask=delta > 0
            r_mask=(c_max==r) & mask
            g_mask=(c_max==g) & mask
            b_mask=(c_max==b) & mask

            # Aplică formulele pentru calculul nuanței în funcție de canalul dominant
            h[r_mask]=(60*((g[r_mask]-b[r_mask])/delta[r_mask])+360)%360
            h[g_mask]=(60*((b[g_mask]-r[g_mask])/delta[g_mask])+120)%360
            h[b_mask]=(60*((r[b_mask]-g[b_mask])/delta[b_mask])+240)%360
            h[h<0]+=360

            # Calculează componentele S (Saturation) și V (Value)
            s=np.zeros_like(c_max,dtype=np.float32)
            s[c_max>0]=delta[c_max>0]/c_max[c_max>0]
            v=c_max
            
            # Convertește la formatul OpenCV (H: 0-179, S: 0-255, V: 0-255)
            h=(h/2).astype(np.uint8)  # H în OpenCV este 0-179
            s=(s*255).astype(np.uint8)
            v=(v*255).astype(np.uint8)

            # Combină canalele în imagine HSV
            current_image=np.stack([h,s,v],axis=-1)
            
        case 6:  # HSL OpenCV
            current_image=cv2.cvtColor(current_image,cv2.COLOR_BGR2HLS)
            
        case 7:  # HSL calculat manual
            # Inițializează variabile pentru calcul
            hsl_computed=current_image.astype(np.float32)/255.0
            r,g,b=original_image[...,0],original_image[...,1],original_image[...,2]
            cmax=np.max(original_image, axis=-1)
            cmin=np.min(original_image, axis=-1)
            delta=cmax-cmin
            
            # Calculează L (Lightness)
            l=(cmax+cmin)/2

            # Calculează S (Saturation)
            s=np.zeros_like(l)
            mask=delta>0
            s[mask]=delta[mask]/(1-np.abs(2*l[mask]-1))
        
            # Calculează H (Hue)
            h=np.zeros_like(cmax, dtype=np.float32)
            r_mask=(cmax==r) & mask
            g_mask=(cmax==g) & mask
            b_mask=(cmax==b) & mask
        
            h[r_mask]=(60*((g[r_mask]-b[r_mask])/delta[r_mask])% 6)
            h[g_mask]=(60*((b[g_mask]-r[g_mask])/delta[g_mask])+2)
            h[b_mask]=(60*((r[b_mask]-g[b_mask])/delta[b_mask])+4)
            h[h<0]+=360  # Asigură valori pozitive
        
            # Convertește la format OpenCV
            h=(h/2).astype(np.uint8)  # Scalează H la [0, 179]
            s=(s*255).astype(np.uint8)  # Scalează S la [0, 255]
            l=(l*255).astype(np.uint8)  # Scalează L la [0, 255]

            # Combină canalele în imagine HSL
            current_image=np.stack([h,s,l],axis=-1)
            
        case 8:  # YUV
            current_image=cv2.cvtColor(current_image,cv2.COLOR_BGR2YUV)
            
        case 9:  # YCrCb
            current_image=cv2.cvtColor(current_image,cv2.COLOR_BGR2YCrCb)
            
        case 10:  # CMYK 
            # Convertește BGR la interval [0,1]
            bgr=current_image.astype(float)/255.0
            # Calculează componentele CMYK
            k=1-np.max(bgr,axis=2)
            c=(1-bgr[...,2]-k)/(1-k+0.00001)  # Evită împărțirea la zero
            m=(1-bgr[...,1]-k)/(1-k+0.00001)
            y=(1-bgr[...,0]-k)/(1-k+0.00001)
            
            # Convertește înapoi la format BGR pentru afișare
            cmyk=np.zeros_like(current_image,dtype=np.float32)
            cmyk[...,0]=255*c
            cmyk[...,1]=255*m
            cmyk[...,2]=255*y
            current_image=cmyk.astype(np.uint8)
            
        case 11:  # LAB
            current_image=cv2.cvtColor(current_image,cv2.COLOR_BGR2LAB)
            
        case _:  # Default case
            pass
            
    # Actualizează afișarea cu noua imagine procesată
    update_image_display()

## 4. Funcțiile pentru încărcarea, setarea extensiei și salvarea imaginilor

In [92]:
def load_image():
    """Încarcă o imagine din sistemul de fișiere."""
    global current_image, original_image, current_path
    
    # Creează un dialog pentru selectarea fișierelor
    file_dialog=QFileDialog()
    
    # Deschide fereastra de dialog pentru alegerea unui fișier imagine
    file_path,_=file_dialog.getOpenFileName(main_window,"Încarcă imagine","","Fișiere imagine (*.png *.jpg *.jpeg *.bmp *.webp *.jfif)")
    
    if file_path:
        # Salvează calea absolută către fișier
        current_path=os.path.abspath(file_path)
        
        # Încarcă imaginea folosind OpenCV
        original_image=cv2.imread(current_path,cv2.IMREAD_UNCHANGED)
        if original_image is None:
            return
        
        # Creează o copie pentru prelucrare
        current_image=original_image.copy()
        # Actualizează imaginea în interfață
        update_image_display()

In [93]:
def set_selected_format(ext):
    """Setează formatul de fișier ales.
        @param 1 ext - în acest parametru se salvează extensia pe care o va avea imaginea în momentul salvării
    """
    global selected_format  # Ne vom folosi de variabila globală selected_format, în favoarea unei variabile locale funcției

    # Actualizează valoarea variabilei globale cu extensia primită ca parametru în cadrul funcției
    selected_format=ext # Această extensie va fi folosită ulterior la salvarea imaginii

In [94]:
def save_image():
    """Salvează imaginea curentă în formatul selectat."""
    global current_image,selected_format

    # Verifică dacă există o imagine și un format selectat
    if current_image is None or not selected_format:
        return # Funcția se oprește dacă nu avem ce salva sau unde

    # Deschide un dialog de tip "Save file" din Qt pentru a permite utilizatorului să aleagă locația și numele fișierului pe care îl salvează
    file_dialog=QFileDialog()
    file_path,_=file_dialog.getSaveFileName(main_window,"Salvează imaginea","",f"Fișiere imagine (*{selected_format})")

    # Verifică dacă utilizatorul nu a anulat operațiunea
    if file_path:
        
        # Ne asigurăm că extensia este corectă și vom adauga extensia dacă utilizatorul a omis-o în procesul de salvare
        if not file_path.endswith(selected_format):
            file_path+=selected_format
        
        # Salvează imaginea folosind OpenCV (cv2) în locația specificată
        # Presupunem că current_image este o matrice compatibilă cu OpenCV
        cv2.imwrite(file_path,current_image)

## 5. Funcția pentru actualizarea imaginii locale cu modificările efectuate

In [95]:
def update_image_display():
    """Actualizează afișarea imaginii în interfață."""
    global current_image, image_label
    
    # Verifică dacă există o imagine încărcată
    if current_image is None:
        return
        
    # Convertește imaginea din BGR la RGB (OpenCV folosește BGR)
    rgb_image=cv2.cvtColor(current_image,cv2.COLOR_BGR2RGB)
    
    # Obține dimensiunile și canalele imaginii
    height,width,channels=rgb_image.shape
    
    # Calculează numărul de bytes per linie
    bytes_per_line=channels*width
    
    # Creează un obiect QImage din datele imaginii
    q_image=QImage(rgb_image.data,width,height,bytes_per_line,QImage.Format_RGB888)
    
    # Convertește QImage în QPixmap pentru afișare în QLabel
    pixmap=QPixmap.fromImage(q_image)
    
    # Setează pixmap-ul la eticheta de imagine din interfață
    image_label.setPixmap(pixmap)
    
    # Ajustează dimensiunea etichetei pentru a se potrivi cu imaginea
    image_label.adjustSize()

## 6. Funcțiile pentru editarea imaginii

## 6.1 Funcția pentru ajustarea brightness-ului și a contrastului

In [96]:
def adjust_brightness_contrast():
    """Ajustează luminozitatea și contrastul imaginii implementând manual formulele."""
    global current_image, original_image, brightness_slider, contrast_slider
    
    if original_image is None:
        return

    # Se vor lua valorile de pe slidere atât pentru contrast, dar și pentru brightness
    brightness=brightness_slider.value()
    contrast=contrast_slider.value()/100.0
    
    # Creăm o copie a imaginii originale pentru a nu o modifica
    current_image=original_image.copy()
    
    # Formula: pixel_nou=contrast*pixel_original+brightness
    # Obținem dimensiunile imaginii
    height,width=current_image.shape[:2]
    
    # Parcurgem fiecare pixel din imagine
    for y in range(height):
        for x in range(width):
            # Pentru fiecare canal de culoare (B, G, R)
            for c in range(current_image.shape[2]):
                # Aplicăm formula pentru ajustarea luminozității și contrastului
                # contrast * pixel + brightness
                pixel_value=current_image[y,x,c]
                new_value=contrast*pixel_value+brightness
                
                # Asigurăm că valorile rămân în intervalul valid [0, 255]
                new_value=max(0,min(255,new_value))
                
                # Actualizăm valoarea pixelului
                current_image[y,x,c]=new_value
    
    # Convertim la tipul de date uint8 (0-255) pentru a asigura compatibilitatea
    current_image=current_image.astype('uint8')
    
    # Actualizăm afișarea imaginii
    update_image_display()

## 6.2 Funcția pentru aplicarea diferitelor filtre

## 6.2.1 Funcția care aplică filtrul blur pe imaginea selectată

In [97]:
def apply_blur(image, kernel_size=(5,5)):
    """Implementarea filtrului Blur (Box filter)
    Intrări
        @ param1 image: Imaginea selectata
        @ param2 kernel_size: Dimensiunea kernel-ului pentru filtrare (lățime, înălțime)
    
    Returnează:
        Imaginea în urma aplicării filtrului
    """
    
    # Verificăm dacă imaginea este grayscale (un singur canal)
    if len(image.shape)==2:
        image=image[:,:,np.newaxis]  # Convertim la 3D pentru procesare uniformă
    
    # Creăm o copie a imaginii pentru rezultat
    result=np.zeros_like(image,dtype=np.float32)  # Folosim float32 pentru calcule
    
    # Extragem dimensiunile imaginii și kernel-ului
    height,width=image.shape[:2]
    k_height,k_width=kernel_size
    
    # Calculăm deplasamentele pentru kernel
    pad_height=k_height//2
    pad_width=k_width//2
    
    # Adăugăm padding la imagine folosind reflectare
    padded_image=np.pad(image.astype(np.float32),  # Convertim la float32
                        ((pad_height, pad_height), 
                         (pad_width, pad_width), 
                         (0, 0)), 
                        mode='reflect')
    
    # Calculăm valoarea pentru normalizare (media)
    kernel_size_total=k_height*k_width
    
    # Parcurgem fiecare pixel din imagine
    for y in range(height):
        for x in range(width):
            # Pentru fiecare canal de culoare
            for c in range(image.shape[2]):
                # Folosim float pentru sumă pentru a evita overflow
                pixel_sum=0.0
                
                # Aplicăm kernel-ul
                for ky in range(k_height):
                    for kx in range(k_width):
                        # Calculăm coordonatele
                        py=y+ky
                        px=x+kx
                        
                        # Adăugăm valoarea la sumă
                        pixel_sum+=padded_image[py,px,c]
                
                # Calculăm media și o atribuim (convertim înapoi la uint8)
                result[y,x,c]=np.clip(pixel_sum/kernel_size_total,0,255)
    
    # Returnăm imaginea în formatul original
    return result.astype(np.uint8).squeeze() if len(image.shape)==2 else result.astype(np.uint8)

# 6.2.2 Funcția care aplică filtrul Gaussian pe imaginea selectată

In [98]:
def apply_gaussian_blur(image,kernel_size=(5,5),sigma=0):
    """Implementarea filtrului Gaussian Blur.
    Intrări:
        @param1 image: Imaginea selectată
        @param 2 kernel_size: Dimensiunea kernel-ului pentru filtrare (lățime, înălțime)
        @sigma: Deviația standard pentru distribuția Gaussiană. Dacă este 0, se calculează automat.
    
    Returnează:
        Imaginea în urma aplicării filtrului
    """
    
    # Calculăm sigma dacă nu este specificat
    if sigma==0:
        sigma=0.3*((kernel_size[0]-1)*0.5-1)+0.8
    
    # Creăm o copie a imaginii pentru rezultat
    result=image.copy()
    
    # Extragem dimensiunile imaginii și kernel-ului
    height,width=image.shape[:2]
    k_height,k_width=kernel_size
    
    # Calculăm deplasamentele pentru kernel
    pad_height=k_height//2
    pad_width=k_width//2
    
    # Adăugăm padding la imagine pentru a gestiona marginile
    padded_image=np.pad(image,((pad_height,pad_height),(pad_width,pad_width),(0,0)),mode='reflect')
    
    # Creăm kernel-ul Gaussian
    gaussian_kernel=np.zeros(kernel_size)
    center_y,center_x=k_height//2,k_width//2
    
    # Calculăm valorile kernel-ului
    kernel_sum=0
    for y in range(k_height):
        for x in range(k_width):
            # Calculăm distanța euclidiana la pătrat față de centru
            dist_sq=(y-center_y)**2+(x-center_x)**2
            
            # Aplicăm formula Gaussian
            gaussian_kernel[y,x]=math.exp(-dist_sq/(2*sigma**2))
            kernel_sum+=gaussian_kernel[y,x]
    
    # Normalizăm kernel-ul
    gaussian_kernel=gaussian_kernel/kernel_sum
    
    # Aplicăm kernel-ul pe imagine
    for y in range(height):
        for x in range(width):
            for c in range(image.shape[2]):
                pixel_sum=0
                
                # Aplicăm kernel-ul
                for ky in range(k_height):
                    for kx in range(k_width):
                        # Calculăm coordonatele pentru pixelul curent în imaginea cu padding
                        py=y+ky
                        px=x+kx
                        
                        # Adăugăm valoarea ponderată la sumă
                        pixel_sum+=padded_image[py,px,c]*gaussian_kernel[ky,kx]
                
                # Atribuim rezultatul pixelului
                result[y,x,c]=int(pixel_sum)
    
    return result

# 6.2.3 Funcția care aplică filtrul median pe imaginea selectată

In [99]:
def apply_median_blur(image, kernel_size=5):
    """Implementarea filtrului Median Blur.
    Intrări:
        @param1 image: Imaginea selectată
        @param2 kernel_size: Dimensiunea kernel-ului pentru filtrare (trebuie să fie impară)
    
    Returnează:
        Imaginea în urma aplicării filtrului
    """
    
    # Asigurăm că kernel_size este impar
    if kernel_size%2==0:
        kernel_size+=1
    
    # Creăm o copie a imaginii pentru rezultat
    result=image.copy()
    
    # Extragem dimensiunile imaginii
    height,width=image.shape[:2]
    
    # Calculăm deplasamentul pentru kernel
    pad=kernel_size//2
    
    # Adăugăm padding la imagine pentru a gestiona marginile
    padded_image=np.pad(image,((pad,pad),(pad,pad),(0,0)),mode='reflect')
    
    # Parcurgem fiecare pixel din imagine
    for y in range(height):
        for x in range(width):
            for c in range(image.shape[2]):
                # Inițializăm lista de valori pentru mediana
                values=[]
                
                # Colectăm valorile din fereastra kernel-ului
                for ky in range(kernel_size):
                    for kx in range(kernel_size):
                        # Calculăm coordonatele pentru pixelul curent în imaginea cu padding
                        py=y+ky
                        px=x+kx
                        
                        # Adăugăm valoarea la lista
                        values.append(padded_image[py,px,c])
                
                # Sortăm valorile și alegem mediana
                values.sort()
                median_value=values[len(values)//2]
                
                # Atribuim mediana pixelului din imaginea rezultat
                result[y,x,c]=median_value
    
    return result

## 6.2.4 Funcția care aplică filtrul bileteral pe imaginea selectată

In [100]:
def apply_bilateral_filter(image, d=9, sigma_color=75, sigma_space=75):
    """Implementarea filtrului Bilateral pentru reducerea zgomotului cu păstrarea marginilor.
    Intrări:
        @param1 image: Imaginea selectată
        @param2 d: Diametrul vecinătății fiecărui pixel
        @param3 sigma_color: Deviația standard în spațiul culorilor
        @param4 sigma_space: Deviația standard în spațiul coordonatelor
    
    Returnează:
        Imaginea în urma aplicării filtrului
    """
    
    # Creăm o copie a imaginii pentru rezultat
    result=image.copy()
    
    # Extragem dimensiunile imaginii
    height,width=image.shape[:2]
    
    # Calculăm deplasamentul pentru kernel
    radius=d//2
    
    # Adăugăm padding la imagine pentru a gestiona marginile
    padded_image=np.pad(image,((radius,radius),(radius,radius),(0,0)),mode='reflect')
    
    # Precalculăm kernel-ul spațial (distanțe) pentru eficiență
    spatial_kernel=np.zeros((2*radius+1,2*radius+1))
    for ky in range(-radius,radius+1):
        for kx in range(-radius,radius+1):
            space_dist=ky**2+kx**2
            spatial_kernel[ky+radius,kx+radius]=math.exp(-space_dist/(2*sigma_space**2))
    
    # Precalculăm exponențialele pentru diferențe de culoare pentru eficiență
    color_diff_exp=np.zeros(256)
    for i in range(256):
        color_diff_exp[i]=math.exp(-(i*i)/(2*sigma_color**2))
    
    # Procesăm fiecare canal de culoare în parte pentru o mai bună organizare a memoriei
    for c in range(image.shape[2]):
        for y in range(height):
            for x in range(width):
                # Valoarea pixelului central
                center_value=padded_image[y+radius, x+radius, c]
                
                # Inițializăm sumele pentru calculul ponderat
                weighted_sum=0
                weight_sum=0
                
                # Aplicăm kernel-ul, dar doar pentru vecinătatea relevantă
                for ky in range(-radius,radius+1):
                    for kx in range(-radius,radius+1):
                        # Calculăm coordonatele pentru pixelul curent în imaginea cu padding
                        py=y+radius+ky
                        px=x+radius+kx
                        
                        # Valoarea pixelului curent
                        pixel_value=padded_image[py,px,c]
                        
                        # Obținem ponderea spațială precalculată
                        space_weight=spatial_kernel[ky+radius, kx+radius]
                        
                        # Calculăm diferența de intensitate și folosim valoarea precalculată
                        color_diff=abs(int(center_value)-int(pixel_value))
                        if color_diff<256:
                            color_weight=color_diff_exp[color_diff]
                        else:
                            # Pentru diferențe mari de culoare, ponderea e practic zero
                            color_weight=0
                        
                        # Ponderea finală
                        weight=space_weight*color_weight
                        
                        # Adăugăm la sumele ponderate
                        weighted_sum+=pixel_value*weight
                        weight_sum+=weight
                
                # Normalizăm și atribuim rezultatul pixelului
                if weight_sum>0:
                    result[y,x,c]=int(weighted_sum/weight_sum)
    
    return result

## 6.2.5 Funcția "main" pentru aplicarea filtrelor

In [101]:
def apply_filter(index):
    """Aplică un filtru selectat pe imagine folosind un dicționar de funcții
    în loc de if-else și implementări manuale pentru filtre.
    
    @param1 index : filtrul pe care dorim să îl aplicăm imaginii selectate
    """
    
    global current_image,original_image
    
    if original_image is None:
        return
        
    current_image=original_image.copy()
    
    # Dicționar de funcții pentru simularea unui "switch case"
    filter_functions={
        0: lambda img: img,                  # Fără filtru - returnează imaginea neschimbată
        1: apply_blur,                       # Blur
        2: apply_gaussian_blur,              # Gaussian Blur
        3: apply_median_blur,                # Median Blur
        4: apply_bilateral_filter            # Bilateral Filter
    }
    
    # Verificăm dacă indexul există în dicționar
    if index in filter_functions:
        # Aplicăm funcția corespunzătoare indexului
        current_image=filter_functions[index](current_image)
    
    update_image_display()

## 6.3 Funcția pentru afișarea histogramei imaginii

In [102]:
def show_histogram():
    """Afișează histograma imaginii curente cu sistem de axe XOY echilibrat
    și accent pe distribuția intensităților pixelilor pe axa OX."""
    
    global current_image
    
    if current_image is None:
        return
    
    # Convertim imaginea în tonuri de gri
    gray=cv2.cvtColor(current_image,cv2.COLOR_BGR2GRAY)
    hist=cv2.calcHist([gray],[0],None,[256],[0, 256])
    
    # Setăm dimensiunea imaginii histogramei (aspect panoramic pentru mai mult spațiu pe OX)
    width,height=1000,600
    hist_img=np.zeros((height,width,3),np.uint8)
    hist_img[:,:]=(30, 30, 30)  # Fundal gri închis pentru contrast mai bun
    
    # Normalizăm histograma pentru a avea o înălțime maximă decentă
    max_height=height-150  # Lăsăm mai mult loc pentru etichete și axe
    cv2.normalize(hist,hist,0,max_height,cv2.NORM_MINMAX)
    
    # Setăm punctele de referință pentru axele X și Y
    x_offset,y_offset=80,height-80  # Mutăm histograma în cadru
    
    # Desenăm axele X și Y
    cv2.line(hist_img,(x_offset,50),(x_offset,y_offset),(255,255,255),2)  # Axă Y
    cv2.line(hist_img,(x_offset,y_offset),(width-40,y_offset),(255,255,255),2)  # Axă X
    
    # Desenăm histograma echilibrată
    bar_width=3  # Lățime fixă bară - asigură vizibilitatea fiecărei intensități
    bar_margin=0  # Spațiu între bare
    
    # Calculăm scala pentru a încadra toate barele în spațiul disponibil
    x_scale=(width-x_offset-40)/256
    
    # Desenăm bare distincte pentru fiecare intensitate
    for i in range(256):
        x=int(x_offset+i*x_scale)  # Scalare pe OX
        bar_height=int(hist[i])
        y=y_offset-bar_height  # Calculăm înălțimea barei
        
        # Colorăm bare în funcție de intensitate pentru vizibilitate mai bună
        # Tranziție de la albastru (0) la verde (128) la roșu (255)
        if i<85:
            color=(255,i*3,0)  # Albastru spre cyan
        elif i<170:
            color=(255-((i-85)*3),255,0)  # Cyan spre galben 
        else:
            color=(0,255-((i-170)*3),(i-170)*3)  # Galben spre roșu
        
        # Desenăm o bară plină cu culoarea corespunzătoare
        cv2.rectangle(hist_img,(x,y_offset),(x+bar_width,y),color,-1)
        
        # Desenăm conturul barei pentru contrast
        cv2.rectangle(hist_img,(x,y_offset),(x+bar_width,y),(255,255,255),1)
    
    # Adăugăm linii de grilă pentru o mai bună vizualizare a frecvențelor
    grid_step=max_height // 5
    for i in range(1,6):
        y=y_offset-grid_step*i
        cv2.line(hist_img,(x_offset,y),(width-40,y),(100,100,100),1,cv2.LINE_AA)
        
        # Adăugăm valoarea frecvenței pe grilă
        freq_value=int((grid_step*i)*np.max(hist)/max_height)
        cv2.putText(hist_img,str(freq_value),(x_offset-70,y+5), 
                   cv2.FONT_HERSHEY_SIMPLEX,0.5,(200,200,200),1,cv2.LINE_AA)
    
    # Marcăm mai multe puncte pe axa OX pentru a evidenția intensitățile
    font=cv2.FONT_HERSHEY_SIMPLEX
    for i in range(0,256,32):  # Marcăm din 32 în 32
        x=int(x_offset+i*x_scale)
        
        # Linie verticală mică pe axa OX pentru marcaj
        cv2.line(hist_img,(x,y_offset),(x,y_offset+5),(255,255,255),1)
        
        # Valoarea intensității
        cv2.putText(hist_img,str(i),(x-10,y_offset+25),font,0.4,(200,200,200),1,cv2.LINE_AA)
    
    # Zonele importante de intensitate
    zone_labels=[
        {"val": 0, "text": "Negru", "color": (100, 100, 255)},
        {"val": 64, "text": "Întuneric", "color": (100, 200, 255)},
        {"val": 128, "text": "Mediu", "color": (100, 255, 100)},
        {"val": 192, "text": "Luminos", "color": (100, 255, 255)},
        {"val": 255, "text": "Alb", "color": (0, 150, 255)}
    ]
    
    # Adăugăm etichetele de zone
    for zone in zone_labels:
        x=int(x_offset+zone["val"]*x_scale)
        cv2.putText(hist_img,zone["text"],(x-20,y_offset+45), 
                   font,0.45,zone["color"],1,cv2.LINE_AA)
    
    # Etichete principale pentru axe
    cv2.putText(hist_img,"Frecventa pixelilor",(x_offset-70,30), 
               font,0.7,(255,255,255),1,cv2.LINE_AA)
    cv2.putText(hist_img, "Intensitatea pixelilor (0-255)",(width//2-100,y_offset+70), 
               font,0.7,(255,255,255),1,cv2.LINE_AA)
    
    # Titlu histogramă
    cv2.putText(hist_img,"Distributia intensitatii pixelilor",(width//2-150,30), 
               cv2.FONT_HERSHEY_DUPLEX,0.8,(255,255,255),1,cv2.LINE_AA)
    
    # Setăm dimensiunea ferestrei OpenCV
    cv2.namedWindow("Histograma",cv2.WINDOW_NORMAL)
    cv2.resizeWindow("Histograma",width, height)
    
    # Afișăm histograma
    cv2.imshow("Histograma",hist_img)

## 6.4 Funcția pentru aplicarea efectului de smooting imaginii selectate

In [103]:
def apply_smoothing():
    """Aplică smoothing (netezire) pe imagine folosind un filtru Gaussian implementat manual."""
    
    global current_image, original_image, smoothing_slider
    
    if original_image is None:
        return
        
    kernel_size=smoothing_slider.value()
    if kernel_size%2==0:
        kernel_size+=1  # Kernel size trebuie să fie impar
    
    # Creăm un kernel Gaussian 1D
    sigma=0.3*((kernel_size-1)*0.5-1)+0.8  # Formula standard pentru sigma
    ax=np.linspace(-(kernel_size-1)/2.,(kernel_size-1)/2.,kernel_size)
    gauss=np.exp(-0.5*np.square(ax)/np.square(sigma))
    kernel=np.outer(gauss,gauss)
    kernel=kernel/np.sum(kernel)  # Normalizare
    
    # Aplicăm convoluția manual pe fiecare canal de culoare
    current_image=original_image.copy()
    if len(current_image.shape)==3:  # Imagine color (BGR)
        smoothed=np.zeros_like(current_image)
        for i in range(3):  # Pentru fiecare canal B, G, R
            smoothed[:,:,i]=convolve2d(current_image[:,:,i],kernel,mode='same',boundary='symm')
    else:  # Imagine grayscale
        smoothed=convolve2d(current_image,kernel,mode='same',boundary='symm')
    
    current_image=smoothed.astype(np.uint8)
    update_image_display()

## 6.5 Funcția pentru aplicarea efectului de sharpening imaginii selectate

In [104]:
def apply_sharpening():
    """Aplică efect de sharpening (accentuare a detaliilor) pe imagine."""
    
    global current_image,original_image  # Accesăm variabilele globale
    
    if original_image is None:  # Verificăm dacă există o imagine încărcată
        return  # Ieșim dacă nu există imagine
        
    # Definim kernelul de sharpening (matrice 3x3)
    # Acest kernel amplifică diferențele între pixeli, accentuând detaliile
    kernel=np.array([[-1, -1, -1],
                      [-1,  9, -1],  # Valoarea centrală mare (9) păstrează intensitatea pixelului original
                      [-1, -1, -1]])
                      
    current_image=original_image.copy()  # Lucrăm pe o copie a imaginii originale
    
    # Aplicăm convoluția cu kernelul folosind OpenCV
    # Parametrul -1 specifică că rezultatul va avea aceeași adâncime ca imaginea de intrare
    current_image=cv2.filter2D(current_image,-1,kernel)
    
    update_image_display()  # Actualizăm afișajul cu noua imagine

## 7. Funcțiile pentru transformarea imaginii

## 7.1 Funcția pentru aplicarea scalării imaginii selectate

In [105]:
def apply_scaling():
    """Redimensionează imaginea după factorul selectat din combo-box."""
    
    global current_image,original_image,scale_combo
    
    if original_image is None:  # Verifică dacă există imagine încărcată
        return
        
    scale_index=scale_combo.currentIndex()  # Obține indexul selectat
    scales=[1.0,0.5,0.25,2.0,4.0]  # Factori disponibili: 100%, 50%, 25%, 200%, 400%
    
    if scale_index<0 or scale_index>=len(scales):  # Validare index
        return
        
    scale=scales[scale_index]  # Factorul de scalare selectat
    
    height,width=original_image.shape[:2]  # Dimensiunile originale
    new_width=int(width*scale)  # Noua lățime calculată
    new_height=int(height*scale)  # Noua înălțime calculată
    
    # Redimensionare cu interpolare potrivită (INTER_AREA pentru micșorare, INTER_LINEAR pentru mărire)
    current_image=cv2.resize(original_image,(new_width,new_height), 
                             interpolation=cv2.INTER_AREA if scale<1 else cv2.INTER_LINEAR)
    
    update_image_display()  # Actualizează afișajul

## 7.2 Funcția pentru aplicarea rotirii imaginii selectate

In [106]:
def apply_rotation():
    """Rotește imaginea după unghiul selectat din combo-box."""
    
    global current_image,original_image,rotation_combo
    
    if original_image is None:  # Verifică existența imaginii
        return
        
    rotation_index=rotation_combo.currentIndex()  # Indexul selecției
    angles=[0,90,180,270]  # Unghiurile disponibile
    
    if rotation_index<0 or rotation_index>=len(angles):  # Validare index
        return
        
    angle=angles[rotation_index]  # Unghiul selectat
    
    if angle==0:  # Caz special pentru 0 grade (imagine originală)
        current_image=original_image.copy()
    else:
        height,width=original_image.shape[:2]  # Dimensiunile imaginii
        center=(width//2,height//2)  # Centrul de rotație
        
        # Generează matricea de rotație
        rotation_matrix=cv2.getRotationMatrix2D(center,angle,1.0)
        
        # Aplică transformarea afina
        current_image=cv2.warpAffine(original_image,rotation_matrix,(width,height))

    update_image_display()  # Actualizează afișajul

## 7.3 Funcția care resetează imaginea selectată la forma standard în fiecare tab din interfață

In [107]:
# Prin imaginea la formă standard, ne referim la imaginea inițială, înainte să fie supusă la operații precum schimbarea spațiului de culoare, 
# schimbarea formatului de imagine și a editării sau a transformării acesteia 
def reset_image():
    """Resetează imaginea la starea originală și resetează controalele."""
    
    global current_image,original_image,brightness_slider,contrast_slider,filter_combo,smoothing_slider
    
    if original_image is None:  # Verifică dacă există imagine încărcată
        return
        
    current_image=original_image.copy()  # Restaurează imaginea originală
    update_image_display()  # Actualizează afișajul
    
    # Resetează toate controalele la valorile implicite
    brightness_slider.setValue(0)       # Luminozitate la valoarea neutră
    contrast_slider.setValue(100)       # Contrast la valoarea normală (100%)
    filter_combo.setCurrentIndex(0)     # Fără filtru aplicat
    smoothing_slider.setValue(1)        # Fără efect de netezire

## 8. Funcțiile pentru instanțierea UI-ului

## 8.1 Funcția pentru crearea UI-ului

In [108]:
def create_ui():
    """Creează și configurează interfața grafică a editorului de imagini."""
    
    # Declarații globale pentru elementele UI
    global app,main_window,image_label,brightness_slider,contrast_slider
    global filter_combo,smoothing_slider,scale_combo,rotation_combo,format_combo
    global tabs,format_tab,control_layout

    # Inițializare aplicație Qt
    app=QApplication(sys.argv)
    
    # Configurare fereastră principală
    main_window=QMainWindow()
    main_window.setWindowTitle('Editor de Imagini')
    main_window.setGeometry(100, 100, 1200, 800)  # Poziție și dimensiuni

    # Widget și layout principal
    main_widget=QWidget()
    main_window.setCentralWidget(main_widget)
    main_layout=QHBoxLayout(main_widget)  # Layout orizontal

    # Splitter pentru zonă imagine/controale
    splitter=QSplitter(Qt.Horizontal)
    main_layout.addWidget(splitter)

    # Configurare zonă afișare imagine
    image_scroll=QScrollArea()  # Cu scroll pentru imagini mari
    image_label=QLabel()  # Label pentru afișarea imaginii
    image_label.setAlignment(Qt.AlignCenter)
    image_scroll.setWidget(image_label)
    splitter.addWidget(image_scroll)

    # Configurare zonă controale
    control_widget=QWidget()
    control_layout=QVBoxLayout(control_widget)  # Layout vertical
    splitter.addWidget(control_widget)
    splitter.setSizes([800, 400])  # Proporții inițiale

    # Buton încărcare imagine
    load_button=QPushButton('Încarcă imagine')
    load_button.clicked.connect(load_image)
    control_layout.addWidget(load_button)

    # Inițializare sistem de tab-uri
    tabs=QTabWidget()  # Grupează controalele pe categorii
    control_layout.addWidget(tabs)

    return main_window  # Returnează fereastra configurată

## 8.2 Funcția pentru crearea tabului pentru schimbarea spațiilor de culoare

In [109]:
def create_color_space_tab():
    """Creează și configurează tab-ul pentru conversii între spații de culoare."""
    
    global tabs  # Accesăm widget-ul global de tab-uri
    
    # Inițializare tab și layout
    color_space_tab=QWidget()  # Crează un nou tab
    color_space_layout=QVBoxLayout()  # Layout vertical pentru butoane
    color_space_tab.setLayout(color_space_layout)

    # Listă de tupluri (text_buton, index_corespunzător)
    color_space_buttons=[
        ('Original', 0),         # 0 - Imaginea originală (BGR)
        ('Grayscale Computed', 1), # 1 - Grayscale calculat manual
        ('Grayscale', 2),        # 2 - Grayscale folosind OpenCV
        ('Computed vs openCV',3), # 3 - Comparație între metodele 1 și 2
        ('HSV', 4),              # 4 - HSV folosind OpenCV
        ('HSV Computed',5),      # 5 - HSV calculat manual
        ('HSL', 6),             # 6 - HSL folosind OpenCV 
        ('HSL Computed',7),      # 7 - HSL calculat manual
        ('YUV', 8),             # 8 - YUV folosind OpenCV
        ('YCrCb', 9),           # 9 - YCrCb folosind OpenCV
        ('CMYK', 10),           # 10 - CMYK calculat
        ('LAB', 11)             # 11 - LAB folosind OpenCV
    ]

    # Creare butoane din listă
    for text,index in color_space_buttons:
        button=QPushButton(text)
        # Conectare la funcția de schimbare spațiu de culoare cu index corespunzător
        button.clicked.connect(lambda _,i=index:change_color_space(i))
        color_space_layout.addWidget(button)  # Adăugare buton în layout

    # Adăugare tab completat la interfața principală
    tabs.addTab(color_space_tab,"1. Spații de culoare")  # Titlu tab

## 8.3 Funcția pentru crearea tabului pentru formatul de imagini

In [110]:
def create_format_tab():
    """Creează și configurează tab-ul pentru salvarea imaginii în diferite formate."""
    global tabs,format_tab,format_buttons  # Accesăm variabilele globale necesare
    
    # Inițializare tab și layout
    format_tab=QWidget()  # Crează un nou tab
    format_layout=QVBoxLayout()  # Layout vertical pentru elemente
    format_tab.setLayout(format_layout)  # Asociază layout-ul tab-ului

    # Etichetă descriptivă pentru secțiune
    format_label=QLabel("Salvează imaginea în formatul:")
    format_label.setMaximumHeight(20)  # Dimensiune fixă pentru etichetă
    format_layout.addWidget(format_label)  # Adaugă eticheta în layout

    # Dicționar cu formatele suportate și extensiile corespunzătoare
    formats={
        "BMP": ".bmp",   # Format BMP (necomprimat)
        "JPEG": ".jpg",  # Format JPEG (comprimat cu pierderi)
        "PNG": ".png",   # Format PNG (comprimat fără pierderi)
        "WEBP": ".webp", # Format WebP (comprimare modernă)
        "SVG": ".svg"    # Format vectorial SVG
    }
    
    format_buttons={}  # Dicționar pentru stocarea butoanelor

    # Creare butoane pentru fiecare format
    for fmt,ext in formats.items():
        button=QPushButton(fmt)  # Crează buton cu numele formatului
        # Conectează butonul la funcția set_selected_format cu parametrul ext
        button.clicked.connect(partial(set_selected_format, ext))
        format_layout.addWidget(button)  # Adaugă butonul în layout
        format_buttons[ext]=button  # Stochează butonul în dicționar

    # Buton principal pentru salvarea imaginii
    save_button=QPushButton("Salvează imaginea")
    save_button.clicked.connect(save_image)  # Conectează la funcția de salvare
    format_layout.addWidget(save_button)  # Adaugă butonul în layout

    # Adaugă tab-ul completat la interfața principală
    tabs.addTab(format_tab,"2. Formate de imagine")

## 8.4 Funcția pentru crearea tabului pentru editarea imaginii

In [111]:
def create_edit_tab():
    """Creează tab-ul cu instrumente pentru editarea imaginilor (luminozitate, contrast, filtre etc.)."""
    
    global tabs,brightness_slider,contrast_slider,filter_combo,smoothing_slider
    
    # Inițializare tab și layout principal
    edit_tab=QWidget()
    edit_layout=QVBoxLayout()
    edit_tab.setLayout(edit_layout)

    # ------------------- Secțiune Luminozitate -------------------
    
    brightness_group=QGroupBox("Brightness")
    brightness_layout=QVBoxLayout()
    brightness_group.setLayout(brightness_layout)

    brightness_slider=QSlider(Qt.Horizontal)
    brightness_slider.setMinimum(-100)   # Valoare minimă (-100%)
    brightness_slider.setMaximum(100)    # Valoare maximă (+100%)
    brightness_slider.setValue(0)        # Valoare implicită (neutră)
    brightness_slider.setTickPosition(QSlider.TicksBelow)
    brightness_slider.setTickInterval(10)
    brightness_slider.valueChanged.connect(adjust_brightness_contrast)  # Conectare la funcția de ajustare
    brightness_layout.addWidget(brightness_slider)

    edit_layout.addWidget(brightness_group)

    # ------------------- Secțiune Contrast -------------------
    
    contrast_group=QGroupBox("Contrast")
    contrast_layout=QVBoxLayout()
    contrast_group.setLayout(contrast_layout)

    contrast_slider=QSlider(Qt.Horizontal)
    contrast_slider.setMinimum(0)       # 0% contrast
    contrast_slider.setMaximum(200)     # 200% contrast
    contrast_slider.setValue(100)       # 100% contrast (normal)
    contrast_slider.setTickPosition(QSlider.TicksBelow)
    contrast_slider.setTickInterval(10)
    contrast_slider.valueChanged.connect(adjust_brightness_contrast)  # Folosește aceeași funcție
    contrast_layout.addWidget(contrast_slider)

    edit_layout.addWidget(contrast_group)

    # ------------------- Secțiune Filtre -------------------
    
    filter_group=QGroupBox("Filtre")
    filter_layout=QVBoxLayout()
    filter_group.setLayout(filter_layout)

    filter_combo=QComboBox()
    filter_combo.addItems([
        "Fără filtru",          # Index 0
        "Blur",                # Index 1
        "Gaussian Blur",       # Index 2
        "Median Blur",         # Index 3
        "Bilateral Filter"    # Index 4
    ])
    filter_combo.currentIndexChanged.connect(apply_filter)  # Conectare la funcția de filtrare
    filter_layout.addWidget(filter_combo)

    edit_layout.addWidget(filter_group)

    # ------------------- Secțiune Histogramă -------------------
    
    histogram_button=QPushButton("Afișează histograma")
    histogram_button.clicked.connect(show_histogram)  # Conectare la funcția de afișare histogramă
    edit_layout.addWidget(histogram_button)

    # ------------------- Secțiune Netezire -------------------
    
    smoothing_group=QGroupBox("Smoothing")
    smoothing_layout=QVBoxLayout()
    smoothing_group.setLayout(smoothing_layout)

    smoothing_slider=QSlider(Qt.Horizontal)
    smoothing_slider.setMinimum(1)      # Valoare minimă (fără efect)
    smoothing_slider.setMaximum(25)     # Valoare maximă (blur puternic)
    smoothing_slider.setValue(1)        # Valoare implicită
    smoothing_slider.setTickPosition(QSlider.TicksBelow)
    smoothing_slider.setTickInterval(2)
    smoothing_slider.valueChanged.connect(apply_smoothing)  # Conectare la funcția de netezire
    smoothing_layout.addWidget(smoothing_slider)

    edit_layout.addWidget(smoothing_group)

    # ------------------- Secțiune Sharpening -------------------
    
    sharpening_button=QPushButton("Aplică sharpening")
    sharpening_button.clicked.connect(apply_sharpening)  # Conectare la funcția de accentuare
    edit_layout.addWidget(sharpening_button)

    # Adăugare tab la interfața principală
    tabs.addTab(edit_tab,"3. Editare imagini")

## 8.5 Funcția pentru crearea tabului pentru transformarea imaginii

In [112]:
def create_transform_tab():
    """Creează tab-ul pentru operații de transformare a imaginilor (scalare și rotație)."""
    
    global tabs,scale_combo,rotation_combo  # Accesăm variabilele globale necesare
    
    # Inițializare tab și layout principal
    transform_tab=QWidget()
    transform_layout=QVBoxLayout()
    transform_tab.setLayout(transform_layout)

    # ------------------- Secțiune Scalare -------------------
    
    scaling_group=QGroupBox("Scalare")
    scaling_layout=QVBoxLayout()
    scaling_group.setLayout(scaling_layout)

    # Combobox pentru selectarea factorului de scalare
    scale_combo=QComboBox()
    scale_combo.addItems([
        "100%",  # Dimensiune originală (1.0x)
        "25%",   # Reducere la 1/4 (0.25x)
        "50%",   # Reducere la jumătate (0.5x)
        "200%",  # Mărire la dublu (2.0x)
        "400%"   # Mărire la 4x (4.0x)
    ])
    scaling_layout.addWidget(scale_combo)

    # Buton de aplicare scalare
    apply_scale_button=QPushButton("Aplică scalare")
    apply_scale_button.clicked.connect(apply_scaling)  # Conectare la funcția de scalare
    scaling_layout.addWidget(apply_scale_button)

    transform_layout.addWidget(scaling_group)

    # ------------------- Secțiune Rotație -------------------
    
    rotation_group=QGroupBox("Rotație")
    rotation_layout=QVBoxLayout()
    rotation_group.setLayout(rotation_layout)

    # Combobox pentru selectarea unghiului de rotație
    rotation_combo=QComboBox()
    rotation_combo.addItems([
        "0°",    # Nicio rotație
        "90°",   # Rotație 90° în sens orar
        "180°",  # Rotație 180°
        "270°"   # Rotație 270° (sau 90° în sens antiorar)
    ])
    rotation_layout.addWidget(rotation_combo)

    # Buton de aplicare rotație
    apply_rotation_button=QPushButton("Aplică rotație")
    apply_rotation_button.clicked.connect(apply_rotation)  # Conectare la funcția de rotație
    rotation_layout.addWidget(apply_rotation_button)

    transform_layout.addWidget(rotation_group)

    # Adăugare tab completat la interfața principală
    tabs.addTab(transform_tab,"4. Transformări")

## 8.6 Funcția pentru adăugarea butonului de reset în interfață

In [113]:
def finalize_ui():
    """Completează interfața grafică prin adăugarea butonului de resetare a imaginii."""
    
    global control_layout  # Accesăm layout-ul global al controalelor
    
    # Buton pentru resetarea imaginii la starea originală
    reset_button=QPushButton("Resetează imaginea")
    reset_button.clicked.connect(reset_image)  # Conectare la funcția de reset
    control_layout.addWidget(reset_button)  # Adăugare buton în layout-ul principal
    
    # Notă: Butonul va apărea în partea de jos a interfeței,
    # după toate celelalte elemente de control adăugate anterior

## 8.7 Funcția pentru inițializarea interfeței

In [114]:
def initialize_ui():
    """Inițializează și asamblează toate componentele interfeței grafice.
    
    Returnează:
        QMainWindow: Fereastra principală a aplicației configurată complet
    """
    # Creează fereastra principală și structura de bază a UI
    window=create_ui()
    
    # Adaugă toate tab-urile principale:
    create_color_space_tab()    # Tab pentru conversii spații de culoare
    create_format_tab()         # Tab pentru operații cu formate de fișiere
    create_edit_tab()           # Tab pentru instrumente de editare
    create_transform_tab()      # Tab pentru transformări geometrice
    
    # Adaugă elementele finale (butoane globale etc.)
    finalize_ui()
    
    # Returnează fereastra principală complet configurată
    return window

## 8.8 Funcția pentru lansarea în funcțiune a aplicației

In [115]:
def run_app():
    """Pornește și rulează aplicația de editare imagini.
    Inițializează interfața, afișează fereastra principală și pornește bucla de evenimente.
    """
    # Inițializează toate componentele interfeței
    window=initialize_ui()
    
    # Afișează fereastra principală
    window.show()
    
    # Pornește bucla principală de evenimente Qt
    sys.exit(app.exec_())

if __name__ == '__main__':
    """Punctul de intrare principal al aplicației."""
    try:
        # Încearcă să ruleze aplicația principală
        run_app()  # Lansează inițializarea UI și bucla de evenimente Qt
        
    except SystemExit:
        # Captează și ignoră excepția SystemExit generată de sys.exit()
        pass  # Necesar pentru a preveni afișarea mesajului în consolă
        
        # Motivul: când utilizatorul apasă X pe fereastră, Qt apelează sys.exit(0),
        # care ar fi generat un mesaj de SystemExit în Jupyter/IPython
        # Acest bloc try-except asigură o închidere "curată"

## 9. Concluzie si Bibliografie
Acum că ați ajuns la final, cam acesta este proiectul nostru interactiv care prezintă atât principalele spații de culori folosite în industrie, cât șo diferite formate în care imaginile sunt salvate în computere, dar și diferite operații de bază în procesarea de imagini. Bineînțeles, functionalitatile pot fi extinse prin implementarea exercițiilor propuse spre rezolvare sau prin adaugarea unor noi funcționaități care să dezvolte aplicația. Proiectul oferă o bază solida pentru înțelegerea atât a spațiilor de culoare, a formatelor de imagini, dar și a operațiilor de prelucrare a imaginilor.
Bibliografie:

* **PyQt5** Documentation: https://riverbankcomputing.com/software/pyqt/intro
* **NumPy** Documentation: https://numpy.org/doc/stable/
* **OpenCV** Documentation: https://docs.opencv.org/4.x/index.html
* Cartea **Practical Python and OpenCV** by Adrian Rosebrock https://minhtn1.github.io/Practical%20Python%20and%20OpenCV,%203rd%20Edition.pdf
